### Binary classification | Outline

1. Load the data
2. Explore and plot the data
3. Transform the data
4. Train the model
5. Evaluate the model
6. Saving the model

In [1]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML,1.4.0-preview"  
    
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;

Installing package Microsoft.ML, version 1.4.0-preview.....................done!

Successfully added reference to package Microsoft.ML, version 1.4.0-preview

Installing package XPlot.Plotly, version 2.0.0.......done!

Successfully added reference to package XPlot.Plotly, version 2.0.0

In [50]:
using Microsoft.ML.Data;

public class ProductReview 
{   
    [LoadColumn(0)]
    public bool Sentiment;
       
    [LoadColumn(1)]
    public string Review;     
}

In [51]:
var dataPath = "./Datasets/Binary-Classification Sentiment/product_reviews.csv";

var mlContext = new MLContext(seed: 1);

IDataView dataView = mlContext.Data.LoadFromTextFile<ProductReview>(dataPath, hasHeader: true, separatorChar: ',');

#### Data exploration

In [52]:
var rows = mlContext.Data.CreateEnumerable<ProductReview>(dataView, reuseRowObject: false)
                    .Take(5)
                    .ToList();

display(rows);

index,Sentiment,Review
0,True,I like the item pricing. My granddaughter wanted to mark on it but I wanted it just for the letters.
1,True,Love the magnet easel... great for moving to different areas... Wish it had some sort of non skid pad on bottom though...
2,True,"""Both sides are magnetic. A real plus when you're entertaining more than one child. The four-year old can find the letters for the words"
3,True,"""Bought one a few years ago for my daughter and she loves it"
4,True,I have a stainless steel refrigerator therefore there are not much space for my son to play with his magnet. Brought this for him to put his magnet on. He enjoys sticking his magnet on it. Great to have so he can play with his alphabet magnets.


#### Plotting

In [54]:
var countPositiveSentiment = dataView.GetColumn<bool>("Sentiment").Count(x => x == true);
var countNegativeSentiment = dataView.GetColumn<bool>("Sentiment").Count(x => x == false);

var values = new List<int> { countPositiveSentiment, countNegativeSentiment};
var labels = new List<string> { "Positive", "Negative" };

var pieChart = Chart.Plot(new Graph.Pie 
{
    labels = labels,
    values = values
});

pieChart

#### Split our data

In [55]:
var trainTestSplit = mlContext.Data.TrainTestSplit(dataView);

#### Data Transformation

In [56]:
var dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText("Review")
        .Append(mlContext.Transforms.CopyColumns("Features", "Review"))
        .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"));

In [57]:
var trainer = mlContext.BinaryClassification.Trainers.AveragedPerceptron(labelColumnName: "Sentiment", numberOfIterations: 10, featureColumnName: "Features");

var trainingPipeline = dataProcessPipeline.Append(trainer);

##### Train the model

In [58]:
ITransformer model = trainingPipeline.Fit(trainTestSplit.TrainSet);

#### Evaluate the model

In [59]:
var predicitions = model.Transform(trainTestSplit.TestSet);
var metrics = mlContext.BinaryClassification.Evaluate(predicitions, labelColumnName: "Sentiment");

metrics
//display("Accuracy:" + " " + metrics.MicroAccuracy)

Unhandled Exception: Probability column 'Probability' not found (Parameter 'schema')

#### Save the model

In [60]:
mlContext.Model.Save(model, trainTestSplit.TrainSet.Schema, "sentimentClassifier.zip");